<div style='background-image: url("../../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computational Seismology</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">The (nodal) Discontinuous Galerkin Method for the elastic wave equation</div>
        </div>
    </div>
</div>

In [ ]:
# This is a configuration step. Please run it before the simulation code!

# Imports etc
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt

# Show the plots in the Notebook.
plt.switch_backend("nbagg")

# import of DG modules from same directory as the notebook
# we use the same functions that we already know from the spectral element method
from gll import *
from lagrange import *
from lagrange1st import *
from legendre import *


# ---------------------------------------------------------------
#
# Discontinous galerkin method for the
# 1D elastic wave equation using an upwind flux
#

# ---------------------------------------------------------------
# Initialization of setup
# ---------------------------------------------------------------
xmax = 1000      # Length of domain
rho = 2500.         # Density for homogeneous model, if you change the density, take that into account for a analytical solution!
vs = 2500
mu = rho*vs**2         #  
N = 2           # Order of Lagrange polynomials
ne = 800        # Number of elements


# ----------------------------------------------------------------

# Initialization of GLL points integration weights
[xi, w] = gll(N)  # xi -> N+1 coordinates [-1 1] of GLL points
                  # w integration weights at GLL locations
    
# Space domain
le = xmax / ne  # Length of elements for regular grid points


# Vector with GLL points, same values for boundary points
k = -1
xg = np.zeros((N +1)* ne)
#xg[k] = 0
for i in range(1, ne + 1):
    for j in range(0, N+1):
        k += 1
        xg[k] = (i - 1) * le + .5 * (xi[j] + 1) * le

x=np.reshape(xg,(ne,(N+1))) #Matrix containing all GLL points in every element
x=x.T


# ---------------------------------------------------------------
# Calculation if time step according to Courant criterion

dxmin = min(np.diff(xg[0:N]))
eps = 0.2    # Courant value

dt = eps * dxmin / vs  # Global time step

#-----------------------------------------------------------------
#---Exercice 3: Find the courant limit, dt fixed and N decreasing
#dt=0.0004 #or some other max. timestep
#eps= dt*(np.abs(mu)/dxmin)
#print(eps)
#----------------------------------------------------------------


J = le / 2  # Mapping - Jacobian, same for every element if the element size is everywhere the same
Ji = 1 / J  # Inverse Jacobian

# Initialization of 1st derivative of Lagrange polynomials used in the stiffness matrix
l1d = lagrange1st(N)  # Array with GLL as columns for each N+1 polynomial


#Low storage Runge-Kutta coefficients
rk4a = [0.0,
-567301805773.0/1357537059087.0,
-2404267990393.0/2016746695238.0,
-3550918686646.0/2091501179385.0,
-1275806237668.0/842570457699.0]

rk4b = [ 1432997174477.0/9575080441755.0,
5161836677717.0/13612068292357.0,
1720146321549.0/2090206949498.0,
3134564353537.0/4481467310338.0,
2277821191437.0/14882151754819.0]

In [ ]:
# -----------------------------------------------------------------
# Initialization of system matrices
# -----------------------------------------------------------------


# variables for elemental matrices
Me = np.zeros(N + 1, dtype=float)
Se = np.zeros([N + 1, N + 1], dtype=float)


# Mass matrix
# Elemental
for i in range(-1, N):
    Me[i + 1] =  w[i + 1] * J #only a vector since the matrix is diagonal



# Build inverse matrix
Minv = np.identity(N+1)
for i in range(0, N+1):
    Minv[i,i] = 1. / Me[i]

print('These are the diagonal entries of the Elemental Mass matrix: ')
print(Me)
print('\n')

# ---------------------------------------------------------------
# Stiffness Matrix
# Elemental

for i in range(-1, N):
    for j in range(-1, N):
            Se[i+1,j+1]=  w[j + 1] * l1d[i + 1, j + 1] #What happened to J?


# ---------------------------------------------------------------
print('This is the elemental stiffness matrix:')
print(Se)


In [ ]:
##------------------------------------------------------
# Initialize solution vectors
v=np.zeros(((N+1),ne))
sigma=np.zeros(((N+1),ne))

resv=np.zeros(((N+1),ne))
ressigma=np.zeros(((N+1),ne))

v0 = v
sigma0 = sigma
vnew = v
sigmanew = sigma

sig=20
x0 =500 #center of the impulse
#sigma0 = 0.5*np.exp(-0.4*((x-x0))**2) #initial condition/impulse
sigma0 = np.exp(-1./sig**2*(x-500.0)**2);

c = np.sqrt(mu/rho)

resv = np.zeros(((N+1),ne))
ressigma = np.zeros(((N+1),ne))

vplot = np.reshape(v0,((N+1)*ne),1)
sigmaplot = np.reshape(sigma0,((N+1)*ne),1)


#initialize matrix A
A = np.zeros((2,2))
absA = np.zeros((2,2))

A[0,1]=-mu
A[1,0]=-1./rho
absA[0,0] = c
absA[1,1] = c
Ap = 1/2.0*(A+absA)
An = 1/2.0*(A-absA)

du = np.zeros((ne, N+1,2))

In [ ]:
fig = plt.figure()
plt.ion()
#which value to plot: sigma=0, v=1?
valueplot=1

if valueplot==0:
    line = plt.plot(xg,sigmaplot,color="red",lw=1.5)
    plt.ylabel(' stress ')
else:
    line = plt.plot(xg,vplot,color="blue",lw=1.5)
    plt.ylabel(' velocity ')
plt.xlabel(' x (m)')
plt.show();

#run always previous window for inititalisation step!!!

############### Time extrapolation ####################################
#
# Choose between the Euler scheme and the second order Runge-Kutta scheme
#
method=1 #Euler=0, RK=1
iplot=50 # output interval
nt =2500
####################
##-------Time loop
v=v0
sigma=sigma0

for it in range(1,nt+1):
    
    #Flux calculation
    du = np.zeros((ne, N+1,2))
    # without bounbary elements
    for i in range(1,ne-1):
        du[i,0,:]=np.dot(Ap,(-sigma[N,i-1],-v[N,i-1] )) + np.dot(An,(-sigma[0,i], -v[0,i]))
        du[i,N,:]=np.dot(Ap,(sigma[N,i],v[N,i] )) + np.dot(An,(sigma[0,i+1], v[0,i+1]))
    # boundary elements
        du[0,0,:]=np.dot(Ap,(0,0 )) + np.dot(An,(-sigma[0,0], -v[0,0]))
        du[0,N,:]=np.dot(Ap,(sigma[N,0],v[N,0] )) + np.dot(An,(sigma[0,1], v[0,1]))
        du[ne-1,0,:]=np.dot(Ap,(-sigma[N,ne-2],-v[N,ne-2] )) + np.dot(An,(-sigma[0,ne-1], -v[0,ne-1]))
        du[ne-1,N,:]=np.dot(Ap,(sigma[N,ne-1],v[N,ne-1] )) + np.dot(An,(0, 0))
    # in terms of the FV: Ap*Q(i)+An*Q(i+1) - Ap*Q(i-1) - An*Q(i) = Ap*(Q(i)-Q(i-1)) + An (Q(i+1)-Q(i))   
        
    if method==0: #Euler scheme
        # Extrapolation for every element
        for k in range(0,ne):
               vnew[:,k] = dt* (np.dot(Minv, (-1./rho)*np.dot(Se, sigma[:,k])-du[k,:,1])) + v[:,k]
               sigmanew[:,k] = dt* (np.dot(Minv, (-mu)*np.dot(Se, v[:,k])-du[k,:,0])) + sigma[:,k]

        v = vnew
        sigma = sigmanew
        
    if method==1: #RK scheme
        for INTRK in range(0,5):

            rhsv = np.dot(Minv, -du[:,:,1].T + (-1.0/rho)*np.dot(Se, sigma))
            rhssigma = np.dot(Minv, -du[:,:,0].T + (-mu)*np.dot(Se, v))
            
            resv = rk4a[INTRK]*resv + dt*rhsv
            v = v + rk4b[INTRK]*resv
            
            
            ressigma = rk4a[INTRK]*ressigma + dt*rhssigma
            sigma = sigma + rk4b[INTRK]*ressigma
        
##--Plotting section--###

    if not it % iplot:  

        vplot=np.reshape(v,((N+1)*ne),1)
        sigmaplot=np.reshape(sigma,((N+1)*ne),1)
        for l in line:
            l.remove()
            del l
        if valueplot==0:
            line = plt.plot(xg, sigmaplot, color="red", lw = 1.5)
        else:
            line = plt.plot(xg, vplot, color="blue", lw = 1.5)
        plt.title('Time step : %g ' %(dt*it))
        fig.canvas.draw()

### end of time loop        
####################


In [ ]:
np.shape(du[:,:,1].T)